# Cargamos módulos

In [46]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
import unicodedata

# Datos de Consumo de agua CDMX 2019

In [ ]:
df = pd.read_csv("/home/jfractal/Documentos/dataton_2021/dataton_AGUA_2021/consumo-agua.csv")
#df.dtypes ## muestra los nombres de los campos y su tipo.
df.columns

In [41]:
df2 = df.iloc[:, [8,9,7,17,1]]
#df2.head(10)

In [ ]:
df2['lat'], df2['lon'] = df2['geo_point_2d'].str.split(',', 1).str ## Separamos el campo geo_point_2d en dos nuevas columnas
#df2.head()

In [43]:
consumoAguaAlcaldia = pd.DataFrame(df2.groupby(["alcaldia"]).consumo_total_dom.sum())##Agrupamos por alcaldía y calculamos su total de consumo

consumoAguaAlcaldia = consumoAguaAlcaldia.rename_axis('alcaldia').reset_index()
print(consumoAguaAlcaldia)


               alcaldia  consumo_total_dom
0        ALVARO OBREGON       6.590161e+06
1          AZCAPOTZALCO       6.153755e+06
2         BENITO JUAREZ       9.894931e+06
3              COYOACAN       5.128303e+06
4            CUAJIMALPA       2.064264e+06
5            CUAUHTEMOC       1.034611e+07
6     GUSTAVO A. MADERO       9.196449e+06
7             IZTACALCO       4.663498e+06
8            IZTAPALAPA       5.263020e+06
9   MAGDALENA CONTRERAS       8.449665e+05
10       MIGUEL HIDALGO       8.640463e+06
11           MILPA ALTA       7.250709e+04
12              TLAHUAC       6.018023e+05
13              TLALPAN       2.877269e+06
14  VENUSTIANO CARRANZA       5.007008e+06
15           XOCHIMILCO       1.283415e+06


In [ ]:
consumoAguaColonia = pd.DataFrame(df2.groupby(["colonia"]).consumo_total_dom.sum())##Agrupamos por colonia y calculamos su consumo Total de agua
print(consumoAguaColonia)


In [ ]:
consumoAguaIndiceDes = pd.DataFrame( ((df2.groupby(['indice_des']).consumo_total_dom.sum() )))##Agrupamos por índice de desarrollo y calculamos su consumo total de agua
consumoAguaIndiceDes.head()

In [ ]:
coloniasIndiceDes = pd.DataFrame(df2.indice_des.value_counts() )### Calculamos una tabla de frecuencias de colonias con su indice de desarrollo
print(coloniasIndiceDes)

# Datos de Población CDMX 2020

In [ ]:
pob = pd.read_csv("/home/jfractal/Documentos/dataton_2021/dataton_AGUA_2021/ITER2020_09CDMX.csv")
pob.head(1)

In [44]:
pob.columns
pobT = pob.iloc[:,[3,5,6]]
pobT = pobT.rename(columns={"Nombre del municipio o demarcación territorial": "NomMun", "Nombre de la localidad":"NomLoc", "Población total":"PT"  } )
pobT = pobT[ (pobT['NomLoc'] == "Total del Municipio") ]
pobT = pobT.iloc[:,[0,2]]
print(pobT)

                     NomMun         PT
3              Azcapotzalco   432205.0
5                  Coyoacán   614447.0
7     Cuajimalpa de Morelos   217686.0
34        Gustavo A. Madero  1173351.0
36                Iztacalco   404695.0
38               Iztapalapa  1835486.0
40   La Magdalena Contreras   247622.0
66               Milpa Alta   152685.0
313          Álvaro Obregón   759137.0
322                 Tláhuac   392313.0
377                 Tlalpan   699928.0
544              Xochimilco   442178.0
658           Benito Juárez   434153.0
660              Cuauhtémoc   545884.0
662          Miguel Hidalgo   414470.0
664     Venustiano Carranza   443704.0


# Creamos un dataset de la unión de datos de consumo de agua y población total de la cdmx

In [ ]:
#Homologamos el campo "alcaldia". Transformamos de minusculas a mayusculas y quitamos tíldes
pobT["NomMun"] = pobT["NomMun"].str.upper()##Tranformar de minusculas a mayusculas
pobT['NomMun']=pobT['NomMun'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')##Quita acentos
pobT = pobT.replace({"CUAJIMALPA DE MORELOS": "CUAJIMALPA",
               "LA MAGDALENA CONTRERAS": "MAGDALENA CONTRERAS" })##Remplazamos los registros
print(pobT)

In [63]:
pobT_consAgua = pobT.merge(consumoAguaAlcaldia, left_on='NomMun', right_on='alcaldia')
pobT_consAgua = pobT_consAgua.iloc[:,[0,1,3]]
print(pobT_consAgua)

                 NomMun         PT  consumo_total_dom
0          AZCAPOTZALCO   432205.0       6.153755e+06
1              COYOACAN   614447.0       5.128303e+06
2            CUAJIMALPA   217686.0       2.064264e+06
3     GUSTAVO A. MADERO  1173351.0       9.196449e+06
4             IZTACALCO   404695.0       4.663498e+06
5            IZTAPALAPA  1835486.0       5.263020e+06
6   MAGDALENA CONTRERAS   247622.0       8.449665e+05
7            MILPA ALTA   152685.0       7.250709e+04
8        ALVARO OBREGON   759137.0       6.590161e+06
9               TLAHUAC   392313.0       6.018023e+05
10              TLALPAN   699928.0       2.877269e+06
11           XOCHIMILCO   442178.0       1.283415e+06
12        BENITO JUAREZ   434153.0       9.894931e+06
13           CUAUHTEMOC   545884.0       1.034611e+07
14       MIGUEL HIDALGO   414470.0       8.640463e+06
15  VENUSTIANO CARRANZA   443704.0       5.007008e+06


In [71]:
## Creamos nuevas variables

pobT_consAgua["ConsumoPercapita"] = pobT_consAgua.consumo_total_dom/pobT_consAgua.PT
pobT_consAgua.sort_values(by=['ConsumoPercapita'], inplace=True, ascending=False )
print(pobT_consAgua)

                 NomMun         PT  consumo_total_dom  ConsumoPercapita
12        BENITO JUAREZ   434153.0       9.894931e+06         22.791345
14       MIGUEL HIDALGO   414470.0       8.640463e+06         20.847017
13           CUAUHTEMOC   545884.0       1.034611e+07         18.952952
0          AZCAPOTZALCO   432205.0       6.153755e+06         14.238046
4             IZTACALCO   404695.0       4.663498e+06         11.523487
15  VENUSTIANO CARRANZA   443704.0       5.007008e+06         11.284568
2            CUAJIMALPA   217686.0       2.064264e+06          9.482760
8        ALVARO OBREGON   759137.0       6.590161e+06          8.681123
1              COYOACAN   614447.0       5.128303e+06          8.346209
3     GUSTAVO A. MADERO  1173351.0       9.196449e+06          7.837765
10              TLALPAN   699928.0       2.877269e+06          4.110807
6   MAGDALENA CONTRERAS   247622.0       8.449665e+05          3.412324
11           XOCHIMILCO   442178.0       1.283415e+06          2

# Gráficas

## Gráfica de Consumo de agua por alcaldía

In [58]:
data = [go.Bar(x=consumoAguaAlcaldia["alcaldia"] , y=consumoAguaAlcaldia["consumo_total_dom"]   )]
layout = go.Layout(title="Consumo de Agua Domiciliario por Alcaldía\nCDMX 2019",
                    xaxis= dict(title="Alcaldías"),
                    yaxis= dict(title="Consumo Total de Agua"),
                    barmode = "stack" )
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

## Gráfica de la Población total por alcaldía

In [39]:
data1 = [go.Bar(x=pobT["NomMun"], y=pobT["PT"] )]
layout1 = go.Layout(title="Población Total de la CDMX por Alcaldía 2020", 
                    xaxis=dict(title="Alcaldía"),
                    yaxis=dict(title="Población Total"),
                    barmode="stack" )
fig1 = go.Figure(data=data1, layout=layout1)
pyo.plot(fig1, filename="poblacion_total.html")

'poblacion_total.html'

## Gráfica de Consumo de agua percapita

In [73]:
data2 = [go.Bar(x=pobT_consAgua["NomMun"], y=pobT_consAgua["ConsumoPercapita"])]
layout2 = go.Layout(title="Consumo per cápita de Agua en la CDMX 2019",
                     xaxis=dict(title="Alcaldía"),
                     yaxis=dict(title="Consumo de agua"),
                     barmode="stack")
fig2 = go.Figure(data=data2, layout=layout2)
pyo.plot(fig2, filename="consumo_agua_percapita.html")

'consumo_agua_percapita.html'